In [3]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score
import matplotlib.pyplot as plt
import pickle


In [4]:
# overview the training data
x_train_df = pd.read_csv(os.path.join('data_reviews', 'x_train.csv'))
y_train_df = pd.read_csv(os.path.join('data_reviews', 'y_train.csv'))

tr_text_list = x_train_df.values.tolist()
tr_y_list = y_train_df.values.tolist()

tr_y = np.hstack(np.array(tr_y_list))

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, TensorDataset, random_split
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification


In [6]:
# overview the training data
x_train_df = pd.read_csv(os.path.join('data_reviews', 'x_train.csv'))
y_train_df = pd.read_csv(os.path.join('data_reviews', 'y_train.csv'))

tr_text_list = x_train_df.values.tolist()
tr_y_list = y_train_df.values.tolist()

tr_y = np.hstack(np.array(tr_y_list))

reviews_list = [val[1].lower() for val in tr_text_list]


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokens = tokenizer(reviews_list, padding=True, truncation=True, return_tensors="pt")

# print(tokens)

c:\Users\alyss\miniconda3\envs\cs135_env\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alyss\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [54]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [60]:
dataset = TensorDataset(tokens['input_ids'], tokens['attention_mask'], torch.tensor(tr_y))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)

In [62]:
optimizer = AdamW(model.parameters(), lr=1e-5)

num_loops = 3 # TO DO: experiment with number of loops 
for i in range(num_loops):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
    


/home/rdvaq/miniconda3/envs/cs135_env/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
